In [ ]:
import sleepa as sl
import numpy as np
from numba import njit
import pandas as pd
import h5py

# Interactive plotting stack
import scipy.io as sio
import bokeh_catplot
import bebi103
import colorcet as cc
import hvplot
import hvplot.pandas 
import bokeh.io

import bokeh.layouts
import bokeh.plotting
import holoviews as hv
from bokeh.themes import Theme
from bokeh.plotting import figure
from holoviews.operation import datashader
from holoviews.operation.datashader import rasterize
import matplotlib.pyplot as plt

bokeh.io.output_notebook()
hv.extension('bokeh')

# Set random seed
np.random.seed(34917)

import warnings
warnings.filterwarnings(action = 'ignore')

%load_ext blackcellmagic

In [ ]:
style = sl.get_style_bokeh()


In [ ]:
theme = Theme(json=style)
bokeh.io.curdoc().theme = theme
hv.renderer('bokeh').theme = theme

In [ ]:
# Set report data path 
path =  '../../../prober_lab/data/'

In [ ]:
#Load fish 10 report df. 
fish_10 = pd.read_csv(path + '200501_clus_report_df_fish_10.csv')

In [ ]:
fish_10.head()

In [ ]:
# Extract the cluster labels for each neuron
cluster_labels = fish_10.cluster_labels.values

In [ ]:
n_clusters = max(cluster_labels)
# Make umap plot 
fish_10.sample(1000).hvplot.scatter(
    x = 'UMAP_1', 
    y= 'UMAP_2', 
    c = 'cluster_labels',
    cmap = cc.glasbey_cool[:int(n_clusters)],
    title = 'fish 10',
    clabel = 'clusters',
    alpha = 0.4,
    padding = 0.5,
    width = 500
)

In [ ]:
#'../../../prober_lab/data/subject_10/'

In [ ]:
# Generate and download z_slice 
# sl.get_z_slice(
#     path_to_mat = path + 'subject_10/data_full.mat',
#     data_path = path, 
#     fish_number = 10, 
#     return_df = True
# )

# Generate and download xy_coords
# xy = sl.get_xy_coordinates(
#     path + 'subject_10/data_full.mat',
#     fish_number = 10, 
#     data_path = path, 
#     return_df = True 
# )

In [ ]:
# Load xy coords
xy = pd.read_csv(path + 'xyz_coords_10.csv')

#Load zslice for fish 10
zslice = pd.read_csv(path + 'z_slice_10.csv')

In [ ]:
def plot_cluster_on_brain_plt(z_slice, xy_coords, clus_number, cluster_labels, color): 
    """
    Returns a scatterplot of the neurons in a cluster on top of
    a z-slice of an image the zebrafish brain. 
        
    Params 
    -----
    clus_number(int)
        Number of cluster to plot. 
    cluster_labels(array-like)
        List containing the cluster label for each neuron. 
    color (str)
        Color in HEX format. 
    
    Returns
    --------
    
    fig (matplotlib.figure.Figure)
        Overlay of scatterplot of neurons on image of the brain.
    
    """
    # Initialize figure 
    fig, ax = plt.subplots()
    
    # Plot slice of z-axis of the brain
    ax.imshow(z_slice, cmap = 'bone')

    # Make x and y arrays separately 
    neurons_x = xy_coords[:, 1]
    neurons_y = xy_coords[:, 0]
    
    # Plot neurons scatterplot 
    ax.scatter(neurons_x[cluster_labels == clus_number],neurons_y[cluster_labels == clus_number], s = 0.15,
                alpha = 0.08,color = color, label = 'cluster ' + str(clus_number))
    
    return fig

In [ ]:
xy.head()

In [ ]:
# Generate brain cluster plot
fig = plot_cluster_on_brain_plt(
    zslice.values,
    xy.values,
    clus_number = 1,
    color = cc.glasbey_cool[2],
    cluster_labels= cluster_labels
)

In [ ]:
def brain_cluster_plot_bokeh(z_slice, xy_coords, clus_number, cluster_labels, color):
    """
    Returns a scatterplot of the neurons in a cluster on top of
    a z-slice of an image the zebrafish brain. 
        
    Params 
    -----
    clus_number(int)
        Number of cluster to plot. 

    cluster_labels(array-like)
        List containing the cluster label for each neuron.

    color (str)
        Color in HEX format. 
    
    Returns
    --------
    
    p (bokeh.figure)
        Overlay of scatterplot of neurons on interactive plot
        of the brain.
    """
    
    # Initialize z_slice plot 
    p = bebi103.image.imshow(z_slice, title = 'cluster ' + str(clus_number), cmap = cc.gray)
    
    # Get x and y coordinates
    neurons_x = xy_coords[:, 1]
    neurons_y = xy_coords[:, 0]
    
    # Make scatter plot 
    p.scatter(
        neurons_x[cluster_labels == clus_number],
        neurons_y[cluster_labels == clus_number],
        color = color, 
        size = 1.8, 
        alpha = 0.4
    )
    
    return p

In [ ]:
def get_all_brain_clus_bokeh(
    z_slice, xy_coords, cluster_labels, max_clusters, palette, **kwargs
):

    """
    Wrapper function to plot all clusters with bokeh. 
    
    Params
    ---------
    cluster_labels (array-like or pd.Series)
        Cluster labels for each neuron. 
    
    palette (array-like)
        List of HEX colors. 
    
    Returns
    ---------
    grid (bokeh.models.layouts.Column) 
        Grid of plots. 
    """

    #  Initialize plot list
    plot_list = []

    #  Add individual cluster plot to list
    for ix in np.arange(max_clusters):
    
        plot_list.append(
            brain_cluster_plot_bokeh(
            z_slice,
            xy_coords,
            clus_number=ix,
            color=palette[ix], 
            cluster_labels=cluster_labels
            )
        )
    
    # Generate gridplot 
    grid = bokeh.layouts.gridplot(plot_list, ncols= int(max_clusters //2))

    return grid

In [ ]:
4//2

In [ ]:
neurons_x = xy.values[::100, 1]
neurons_y = xy.values[::100, 0]
    

In [ ]:
zslice.values[:2, :2]

In [ ]:
p = brain_cluster_plot_bokeh(
    zslice.values,
    xy.values,
    clus_number = 1,
    color = cc.glasbey_cool[2],
    cluster_labels= cluster_labels
)

In [ ]:
bokeh.io.show(p)

In [ ]:
# Initialize custom palette 
bright_palette = [
    '#8dd3c7',
    '#762a83', 
    "#1debf2",
    "#fca903",
    "#3df514",
    "#a41df2",
    "#4b00bd",
    "#9aba9e",
    "#95fff2",
    '#e01df2',
    "#14f5f5",
    '#fccde5',
    '#ffffb3'
]

In [ ]:
grid = get_all_brain_clus_bokeh(
    zslice.values,
    xy.values,
    cluster_labels,
    max_clusters = 6,
    palette= bright_palette
)

In [ ]:
bokeh.io.show(grid)